In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from glob import glob
import shutil
import math
import json
import subprocess
from tensorflow.keras.utils import to_categorical

2022-04-13 12:32:43.715248: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/romerik/.local/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-13 12:32:43.715321: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing utilities
mp_holistic = mp.solutions.holistic   #Holistic model

In [3]:
frame_width, frame_height = (256,256)

In [4]:
num_frame_per_video = 25

In [5]:
with open("WLASL/start_kit/WLASL_v0.3.json", "r") as file:
    content = json.load(file)

In [6]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #color conversion
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [7]:
def draw_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [8]:
def draw_styled_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))

In [9]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [10]:
def blank_keypoints():
    pose = np.zeros(132)
    face = np.zeros(1404)
    lh = np.zeros(21*3)
    rh = np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [11]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [12]:
def makedirbyname(path):
    try:
        os.mkdir(path)
    except:
        shutil.rmtree(path)
        os.mkdir(path)

In [13]:
def get_data(train_percent=0.90):
    train_sequences, train_labels = [], []
    test_sequences, test_labels = [], []
    words = glob("Data_Numpy/*")
    for word in words:
        actual_label = word.split("/")[-1]
        videos = glob(word + "/*")
        #print(f"-------------{actual_label}-------")
        train_proportion = math.ceil(round(train_percent * len(videos)))
        for index,video_ in enumerate(videos):
            #print(f"********---> Video   {video_} ----- ")
            window = []
            for frame_num in range(1,num_frame_per_video+1):
                np_file = f"{video_}/{frame_num}.npy"
                #print(np_file)
                res = np.load(np_file)
                window.append(res)
            if index <= train_proportion:
                train_sequences.append(window)
                train_labels.append(label_map[actual_label])
            else:
                test_sequences.append(window)
                test_labels.append(label_map[actual_label])
                
    train_sequences = np.array(train_sequences)
    train_labels = np.array(train_labels)
    test_sequences = np.array(test_sequences)
    test_labels = np.array(test_labels)
                
    return train_sequences, train_labels, test_sequences, test_labels


In [14]:
def get_num_frame(filename):
    video_frame_count = 0
    try:
        cap = cv2.VideoCapture(filename)
        video_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    except:
        pass
    finally:
        cap.release()
    return video_frame_count

In [15]:
def get_duration(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

In [16]:
def create_folders_video(content):
    for i in content:
        folder_path = f"Data/{i['gloss']}"
        createFolder(folder_path)
        for j in i['instances']:
            if os.path.exists(f"WLASL/start_kit/videos/{j['video_id']}.mp4"):
                shutil.copy(f"WLASL/start_kit/videos/{j['video_id']}.mp4", f"Data/{i['gloss']}/{j['video_id']}.mp4")

In [17]:
def get_actions(content):
    actions = []
    for i in content:
        actions.append(i['gloss'])
    return actions

In [18]:
keypoints_blank = blank_keypoints()

In [19]:
keypoints_blank.shape

(1662,)

In [20]:
actions = get_actions(content)

In [21]:
actions = np.array(actions)

In [22]:
actions.shape

(100,)

In [23]:
label_map = {label:num for num,label in enumerate(actions)}

## Generate numpy data for videos

In [ ]:
words = glob("Data/*")
makedirbyname("Data_Numpy")
for word in words:
    videos = glob(word + "/*")
    for video in videos:
        video_path = "/".join("".join(video.split(".")[0:-1]).split("/")[1:])
        createFolder(f"Data_Numpy/{video_path}")
        print(f"Processing collection for... Data_Numpy/{video_path}")
        actual_video_frame = get_num_frame(video)
        if actual_video_frame >= 75:
            remind = actual_video_frame - 75
            to_start = int(remind / 3)
            j = 0
            index = 0
            try:
                cap = cv2.VideoCapture(video)
                with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                    while True:
                        ret, frame = cap.read()
                        j+=1
                        if j < to_start or j%3 != to_start%3:
                            continue
                        
                        index+=1
                        if index > 25:
                            break
                            
                        if ret == False:
                            cap.release()
                            break
                            
                            
                        #Resize the frame
                        resize_frame = cv2.resize(frame, (frame_width, frame_height))

                        # Make detections
                        image, results = mediapipe_detection(resize_frame, holistic)

                        # Drawing landmarks
                        draw_styled_landmark(image, results)

                        #Extracting keypoints
                        keypoints = extract_keypoints(results)
                        
                        if not (keypoints==keypoints_blank).all():
                            last_keypoints = keypoints
                        else:
                            index-=1
                            continue
                            
                        #Saving data
                        np.save(f"Data_Numpy/{video_path}/{index}", keypoints)
                            
                        #cv2.imshow(video, image)
                        if cv2.waitKey(10) and 0xFF == ord("q"):
                            break
                            
                    cap.release()
                    cv2.destroyAllWindows()
            finally:
                while index <=25:
                    np.save(f"Data_Numpy/{video_path}/{index}", last_keypoints)
                    index+=1
                cap.release()
                cv2.destroyAllWindows()
        elif actual_video_frame >=50:
            remind = actual_video_frame - 50
            to_start = int(remind / 3)
            j = 0
            index = 0
            try:
                cap = cv2.VideoCapture(video)
                with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                    while True:
                        ret, frame = cap.read()
                        j+=1

                        if j < to_start or j%2 != to_start%2:
                            continue

                        index+=1
                        if index > 25:
                            break

                        if ret == False:
                            cap.release()
                            break
                        
                        #Resize the frame
                        resize_frame = cv2.resize(frame, (frame_width, frame_height))

                        # Make detections
                        image, results = mediapipe_detection(resize_frame, holistic)

                        # Drawing landmarks
                        draw_styled_landmark(image, results)

                        #Extracting keypoints
                        keypoints = extract_keypoints(results)
                        
                        if not (keypoints==keypoints_blank).all():
                            last_keypoints = keypoints
                        else:
                            index-=1
                            continue
                            
                        #Saving data
                        np.save(f"Data_Numpy/{video_path}/{index}", keypoints)

                        #cv2.imshow(video, image)
                        if cv2.waitKey(10) and 0xFF == ord("q"):
                            break

                    cap.release()
                    cv2.destroyAllWindows()
            finally:
                while index <=25:
                    np.save(f"Data_Numpy/{video_path}/{index}", last_keypoints)
                    index+=1
                cap.release()
                cv2.destroyAllWindows()
        elif actual_video_frame >=25:
            remind = actual_video_frame - 25
            to_start = int(remind / 3)
            j = 0
            index = 0
            try:
                cap = cv2.VideoCapture(video)
                with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                    while True:
                        ret, frame = cap.read()
                        j+=1

                        if j < to_start:
                            continue

                        index+=1
                        if index > 25:
                            break

                        if ret == False:
                            cap.release()
                            break
                            
                        #Resize the frame
                        resize_frame = cv2.resize(frame, (frame_width, frame_height))

                        # Make detections
                        image, results = mediapipe_detection(resize_frame, holistic)

                        # Drawing landmarks
                        draw_styled_landmark(image, results)

                        #Extracting keypoints
                        keypoints = extract_keypoints(results)
                        
                        if not (keypoints==keypoints_blank).all():
                            last_keypoints = keypoints
                        else:
                            index-=1
                            continue
                            
                        #Saving data
                        np.save(f"Data_Numpy/{video_path}/{index}", keypoints)

                        #cv2.imshow(video, image)
                        if cv2.waitKey(10) and 0xFF == ord("q"):
                            break

                    cap.release()
                    cv2.destroyAllWindows()
            finally:
                while index <=25:
                    np.save(f"Data_Numpy/{video_path}/{index}", last_keypoints)
                    index+=1
                cap.release()
                cv2.destroyAllWindows()
        else:
            j = 0
            index = 0
            try:
                cap = cv2.VideoCapture(video)
                with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                    while True:
                        ret, frame = cap.read()
                        j+=1

                        if j < to_start:
                            continue

                        index+=1
                        if index > 25:
                            break

                        if ret == False:
                            cap.release()
                            break
                            
                        #Resize the frame
                        resize_frame = cv2.resize(frame, (frame_width, frame_height))

                        # Make detections
                        image, results = mediapipe_detection(resize_frame, holistic)

                        # Drawing landmarks
                        draw_styled_landmark(image, results)

                        #Extracting keypoints
                        keypoints = extract_keypoints(results)
                        
                        if not (keypoints==keypoints_blank).all():
                            last_keypoints = keypoints
                        else:
                            index-=1
                            continue
                            
                        #Saving data
                        np.save(f"Data_Numpy/{video_path}/{index}", keypoints)

                        #cv2.imshow(video, image)
                        last_frame = frame
                        if cv2.waitKey(10) and 0xFF == ord("q"):
                            break

                    cap.release()
                    cv2.destroyAllWindows()
            finally:
                while index <=25:
                    np.save(f"Data_Numpy/{video_path}/{index}", last_keypoints)
                    #cv2.imshow(video, last_frame)
                    index+=1
                cap.release()
                cv2.destroyAllWindows() 

### Setup data for training

In [27]:
train_sequences, train_labels, test_sequences, test_labels = get_data(0.8)

In [28]:
train_sequences.shape

(1263, 25, 1662)

In [29]:
train_labels.shape

(1263,)

In [30]:
test_sequences.shape

(192, 25, 1662)

In [31]:
test_labels.shape

(192,)

In [32]:
X = train_sequences

In [33]:
X.shape

(1263, 25, 1662)

In [34]:
y = to_categorical(train_labels).astype(int)

In [35]:
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Train the network

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

In [37]:
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [38]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(num_frame_per_video,1662)))
#model.add(LSTM(128, return_sequences=True, activation='relu'))
#model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

2022-04-13 13:28:06.029008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: Ne peut ouvrir le fichier d'objet partagé: Aucun fichier ou dossier de ce type; LD_LIBRARY_PATH: /home/romerik/.local/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-13 13:28:06.029039: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-13 13:28:06.029075: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (romerik): /proc/driver/nvidia/version does not exist
2022-04-13 13:28:06.029856: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [39]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [40]:
model.fit(X, y, epochs=500, batch_size=128,callbacks=[tb_callback])

10/10 [==============================] - 2s 157ms/step - loss: 4.5257 - categorical_accuracy: 0.0166
Epoch 14/500
10/10 [==============================] - 1s 148ms/step - loss: 4.5114 - categorical_accuracy: 0.0198
Epoch 15/500
10/10 [==============================] - 1s 147ms/step - loss: 4.5069 - categorical_accuracy: 0.0214
Epoch 16/500
10/10 [==============================] - 2s 152ms/step - loss: 4.5049 - categorical_accuracy: 0.0182
Epoch 17/500
10/10 [==============================] - 2s 150ms/step - loss: 4.4927 - categorical_accuracy: 0.0230
Epoch 18/500
10/10 [==============================] - 2s 162ms/step - loss: 4.4881 - categorical_accuracy: 0.0293
Epoch 19/500
10/10 [==============================] - 2s 157ms/step - loss: 4.4678 - categorical_accuracy: 0.0277
Epoch 20/500
10/10 [==============================] - 2s 148ms/step - loss: 4.4636 - categorical_accuracy: 0.0206
Epoch 21/500
10/10 [==============================] - 1s 143ms/step - loss: 4.4420 - categorical_accu

Epoch 85/500
10/10 [==============================] - 2s 149ms/step - loss: 3.2025 - categorical_accuracy: 0.2059
Epoch 86/500
10/10 [==============================] - 1s 146ms/step - loss: 3.2035 - categorical_accuracy: 0.1956
Epoch 87/500
10/10 [==============================] - 1s 148ms/step - loss: 3.1243 - categorical_accuracy: 0.2272
Epoch 88/500
10/10 [==============================] - 2s 149ms/step - loss: 3.0979 - categorical_accuracy: 0.2249
Epoch 89/500
10/10 [==============================] - 1s 147ms/step - loss: 3.1498 - categorical_accuracy: 0.2225
Epoch 90/500
10/10 [==============================] - 1s 147ms/step - loss: 3.0975 - categorical_accuracy: 0.2257
Epoch 91/500
10/10 [==============================] - 2s 150ms/step - loss: 3.0352 - categorical_accuracy: 0.2526
Epoch 92/500
10/10 [==============================] - 1s 150ms/step - loss: 3.0381 - categorical_accuracy: 0.2494
Epoch 93/500
10/10 [==============================] - 1s 144ms/step - loss: 3.0886 - cat

10/10 [==============================] - 1s 144ms/step - loss: 2.1215 - categorical_accuracy: 0.4339
Epoch 157/500
10/10 [==============================] - 1s 142ms/step - loss: 2.0467 - categorical_accuracy: 0.4735
Epoch 158/500
10/10 [==============================] - 1s 141ms/step - loss: 2.0824 - categorical_accuracy: 0.4576
Epoch 159/500
10/10 [==============================] - 1s 145ms/step - loss: 2.0021 - categorical_accuracy: 0.4885
Epoch 160/500
10/10 [==============================] - 1s 148ms/step - loss: 1.9367 - categorical_accuracy: 0.4925
Epoch 161/500
10/10 [==============================] - 1s 147ms/step - loss: 1.8771 - categorical_accuracy: 0.5099
Epoch 162/500
10/10 [==============================] - 1s 146ms/step - loss: 1.8672 - categorical_accuracy: 0.5051
Epoch 163/500
10/10 [==============================] - 1s 146ms/step - loss: 1.8948 - categorical_accuracy: 0.4988
Epoch 164/500
10/10 [==============================] - 1s 147ms/step - loss: 1.8454 - categori

10/10 [==============================] - 1s 142ms/step - loss: 1.1591 - categorical_accuracy: 0.6983
Epoch 228/500
10/10 [==============================] - 1s 140ms/step - loss: 1.1585 - categorical_accuracy: 0.6880
Epoch 229/500
10/10 [==============================] - 1s 141ms/step - loss: 1.2210 - categorical_accuracy: 0.6778
Epoch 230/500
10/10 [==============================] - 1s 142ms/step - loss: 1.2276 - categorical_accuracy: 0.6587
Epoch 231/500
10/10 [==============================] - 1s 141ms/step - loss: 1.1659 - categorical_accuracy: 0.6952
Epoch 232/500
10/10 [==============================] - 1s 142ms/step - loss: 1.1958 - categorical_accuracy: 0.6762
Epoch 233/500
10/10 [==============================] - 1s 142ms/step - loss: 1.2405 - categorical_accuracy: 0.6595
Epoch 234/500
10/10 [==============================] - 1s 142ms/step - loss: 1.2972 - categorical_accuracy: 0.6500
Epoch 235/500
10/10 [==============================] - 1s 141ms/step - loss: 1.3401 - categori

10/10 [==============================] - 1s 141ms/step - loss: 0.6884 - categorical_accuracy: 0.8195
Epoch 299/500
10/10 [==============================] - 1s 142ms/step - loss: 0.6496 - categorical_accuracy: 0.8321
Epoch 300/500
10/10 [==============================] - 2s 156ms/step - loss: 0.6798 - categorical_accuracy: 0.8266
Epoch 301/500
10/10 [==============================] - 1s 142ms/step - loss: 0.6513 - categorical_accuracy: 0.8345
Epoch 302/500
10/10 [==============================] - 1s 145ms/step - loss: 0.7332 - categorical_accuracy: 0.8092
Epoch 303/500
10/10 [==============================] - 1s 146ms/step - loss: 0.8312 - categorical_accuracy: 0.7854
Epoch 304/500
10/10 [==============================] - 1s 143ms/step - loss: 1.2676 - categorical_accuracy: 0.6825
Epoch 305/500
10/10 [==============================] - 1s 142ms/step - loss: 1.2980 - categorical_accuracy: 0.6469
Epoch 306/500
10/10 [==============================] - 1s 140ms/step - loss: 1.3073 - categori

10/10 [==============================] - 1s 141ms/step - loss: 0.2879 - categorical_accuracy: 0.9343
Epoch 370/500
10/10 [==============================] - 1s 141ms/step - loss: 0.3433 - categorical_accuracy: 0.9074
Epoch 371/500
10/10 [==============================] - 1s 140ms/step - loss: 0.3718 - categorical_accuracy: 0.9161
Epoch 372/500
10/10 [==============================] - 1s 141ms/step - loss: 0.3354 - categorical_accuracy: 0.9287
Epoch 373/500
10/10 [==============================] - 1s 141ms/step - loss: 0.3110 - categorical_accuracy: 0.9287
Epoch 374/500
10/10 [==============================] - 1s 143ms/step - loss: 0.3047 - categorical_accuracy: 0.9216
Epoch 375/500
10/10 [==============================] - 1s 140ms/step - loss: 0.2943 - categorical_accuracy: 0.9343
Epoch 376/500
10/10 [==============================] - 1s 141ms/step - loss: 0.3802 - categorical_accuracy: 0.9058
Epoch 377/500
10/10 [==============================] - 1s 142ms/step - loss: 1.1990 - categori

10/10 [==============================] - 1s 141ms/step - loss: 0.4142 - categorical_accuracy: 0.9042
Epoch 441/500
10/10 [==============================] - 1s 147ms/step - loss: 0.3126 - categorical_accuracy: 0.9248
Epoch 442/500
10/10 [==============================] - 1s 144ms/step - loss: 0.2533 - categorical_accuracy: 0.9477
Epoch 443/500
10/10 [==============================] - 1s 141ms/step - loss: 0.2386 - categorical_accuracy: 0.9525
Epoch 444/500
10/10 [==============================] - 1s 140ms/step - loss: 0.2024 - categorical_accuracy: 0.9604
Epoch 445/500
10/10 [==============================] - 1s 147ms/step - loss: 0.2042 - categorical_accuracy: 0.9620
Epoch 446/500
10/10 [==============================] - 1s 142ms/step - loss: 0.2062 - categorical_accuracy: 0.9588
Epoch 447/500
10/10 [==============================] - 1s 142ms/step - loss: 0.2316 - categorical_accuracy: 0.9541
Epoch 448/500
10/10 [==============================] - 1s 142ms/step - loss: 0.2576 - categori